##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# CycleGAN

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/generative/cyclegan"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/generative/cyclegan.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/generative/cyclegan.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/generative/cyclegan.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Este notebook demonstra a conversão desemparelhada imagem-para-imagem usando uma GAN condicional, conforme descrito no artigo [Conversão desemparelhada imagem-para-imagem usando Redes Adversárias com Ciclo Consistente](https://arxiv.org/abs/1703.10593), também conhecida como CycleGAN. O artigo propõe um método que consegue capturar as características de um domínio de imagem e descobrir se essas características podem ser convertidas em outro domínio de imagem, tudo sem qualquer exemplo de treinamento emparelhado.

Este notebook pressupõe que você conheça o Pix2Pix. Saiba mais no [tutorial do Pix2Pix](https://www.tensorflow.org/tutorials/generative/pix2pix). O código para a CycleGAN é similar. A principal diferença é uma função de perda adicional e o uso de dados de treinamento desemparelhados.

A CycleGAN usa uma perda de consistência do ciclo para que o treinamento possa ser feito sem a necessidade de se ter dados emparelhados. Em outras palavras, ela pode converter um domínio em outro sem um mapeamento um-para-um entre o domínio fonte e alvo.

Isso abre a possibilidade de se fazer diversas tarefas interessantes, como aprimoramento de fotos, colorização de imagens, transferência de estilos, etc. Você só precisa do dataset fonte e alvo (que é simplesmente um diretório de imagens).

![Imagem da saída 1](https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/generative/images/horse2zebra_1.png?raw=true) ![Imagem da saída 2](https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/generative/images/horse2zebra_2.png?raw=true)

## Configuração do pipeline de entrada

Instale o pacote [tensorflow_examples](https://github.com/tensorflow/examples), que permite a importação do gerador e do discriminador.

In [ ]:
!pip install git+https://github.com/tensorflow/examples.git

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow_datasets as tfds
from tensorflow_examples.models.pix2pix import pix2pix

import os
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output

AUTOTUNE = tf.data.AUTOTUNE

## Pipeline de entrada

Este tutorial treina um modelo para converter imagens de cavalos em imagens de zebras. Este dataset e outros parecidos estão disponíveis [aqui](https://www.tensorflow.org/datasets/catalog/cycle_gan).

Conforme mencionado no [artigo](https://arxiv.org/abs/1703.10593), aplique jitter e espelhamento aleatórios ao dataset de treinamento. Há algumas técnicas de ampliação de imagens que evitam o overfitting.

Isso é similar ao que foi feito no [Pix2Pix](https://www.tensorflow.org/tutorials/generative/pix2pix#load_the_dataset)

- Com jitter aleatório, a imagem é redimensionada para `286x286` e depois cortada aleatoriamente para `256x256`.
- Com espelhamento aleatório, a imagem é invertida horizontalmente de forma aleatória, por exemplo, da esquerda para a direita.

In [ ]:
dataset, metadata = tfds.load('cycle_gan/horse2zebra',
                              with_info=True, as_supervised=True)

train_horses, train_zebras = dataset['trainA'], dataset['trainB']
test_horses, test_zebras = dataset['testA'], dataset['testB']

In [ ]:
BUFFER_SIZE = 1000
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256

In [ ]:
def random_crop(image):
  cropped_image = tf.image.random_crop(
      image, size=[IMG_HEIGHT, IMG_WIDTH, 3])

  return cropped_image

In [ ]:
# normalizing the images to [-1, 1]
def normalize(image):
  image = tf.cast(image, tf.float32)
  image = (image / 127.5) - 1
  return image

In [ ]:
def random_jitter(image):
  # resizing to 286 x 286 x 3
  image = tf.image.resize(image, [286, 286],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

  # randomly cropping to 256 x 256 x 3
  image = random_crop(image)

  # random mirroring
  image = tf.image.random_flip_left_right(image)

  return image

In [ ]:
def preprocess_image_train(image, label):
  image = random_jitter(image)
  image = normalize(image)
  return image

In [ ]:
def preprocess_image_test(image, label):
  image = normalize(image)
  return image

In [ ]:
train_horses = train_horses.cache().map(
    preprocess_image_train, num_parallel_calls=AUTOTUNE).shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)

train_zebras = train_zebras.cache().map(
    preprocess_image_train, num_parallel_calls=AUTOTUNE).shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)

test_horses = test_horses.map(
    preprocess_image_test, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)

test_zebras = test_zebras.map(
    preprocess_image_test, num_parallel_calls=AUTOTUNE).cache().shuffle(
    BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
sample_horse = next(iter(train_horses))
sample_zebra = next(iter(train_zebras))

In [ ]:
plt.subplot(121)
plt.title('Horse')
plt.imshow(sample_horse[0] * 0.5 + 0.5)

plt.subplot(122)
plt.title('Horse with random jitter')
plt.imshow(random_jitter(sample_horse[0]) * 0.5 + 0.5)

In [ ]:
plt.subplot(121)
plt.title('Zebra')
plt.imshow(sample_zebra[0] * 0.5 + 0.5)

plt.subplot(122)
plt.title('Zebra with random jitter')
plt.imshow(random_jitter(sample_zebra[0]) * 0.5 + 0.5)

## Importar e reutilizar os modelos do Pix2Pix

Importe o gerador e o discriminador usados no [Pix2Pix](https://github.com/tensorflow/examples/blob/master/tensorflow_examples/models/pix2pix/pix2pix.py) do pacote [tensorflow_examples](https://github.com/tensorflow/examples) instalado.

A arquitetura do modelo usada neste tutorial é muito similar à usada no [Pix2Pix](https://github.com/tensorflow/examples/blob/master/tensorflow_examples/models/pix2pix/pix2pix.py). Veja algumas diferenças:

- A CycleGAN usa [normalização de instância](https://arxiv.org/abs/1607.08022) em vez de [normalização de lote](https://arxiv.org/abs/1502.03167).
- O [artigo da CycleGAN](https://arxiv.org/abs/1703.10593) usa um gerador baseado em `restnet`. Este tutorial utiliza um gerador `unet` modificado por questões de simplicidade.

Há dois geradores (G e F) e dois discriminadores (X e Y) sendo treinados.

- O Gerador `G` aprende a transformar a imagem `X` na imagem `Y`. $(G: X -> Y)$
- O Gerador `F` aprende a transformar a imagem `Y` na imagem `X`. $(F: Y -> X)$
- O discriminador `D_X` aprende a diferenciar entre a imagem `X` e a imagem gerada `X` (`(F(Y))`).
- O discriminador `D_Y` aprende a diferenciar entre a imagem `Y` e a imagem gerada `Y` (`(G(X))`).

![Modelo da CycleGAN](https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/generative/images/cyclegan_model.png?raw=true)

In [ ]:
OUTPUT_CHANNELS = 3

generator_g = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')
generator_f = pix2pix.unet_generator(OUTPUT_CHANNELS, norm_type='instancenorm')

discriminator_x = pix2pix.discriminator(norm_type='instancenorm', target=False)
discriminator_y = pix2pix.discriminator(norm_type='instancenorm', target=False)

In [ ]:
to_zebra = generator_g(sample_horse)
to_horse = generator_f(sample_zebra)
plt.figure(figsize=(8, 8))
contrast = 8

imgs = [sample_horse, to_zebra, sample_zebra, to_horse]
title = ['Horse', 'To Zebra', 'Zebra', 'To Horse']

for i in range(len(imgs)):
  plt.subplot(2, 2, i+1)
  plt.title(title[i])
  if i % 2 == 0:
    plt.imshow(imgs[i][0] * 0.5 + 0.5)
  else:
    plt.imshow(imgs[i][0] * 0.5 * contrast + 0.5)
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))

plt.subplot(121)
plt.title('Is a real zebra?')
plt.imshow(discriminator_y(sample_zebra)[0, ..., -1], cmap='RdBu_r')

plt.subplot(122)
plt.title('Is a real horse?')
plt.imshow(discriminator_x(sample_horse)[0, ..., -1], cmap='RdBu_r')

plt.show()

## Funções de perda

Na CycleGAN, não há dados emparelhados para fazer o treinamento e, portanto, não há garantia de que o par de entrada `x` e alvo `y` seja significativo durante o treinamento. Dessa forma, para forçar a rede a aprender o mapeamento correto, os autores propõem a perda de consistência do ciclo.

A perda do discriminador e a perda do gerador são similares às usadas no [Pix2Pix](https://www.tensorflow.org/tutorials/generative/pix2pix#build_the_generator).

In [ ]:
LAMBDA = 10

In [ ]:
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def discriminator_loss(real, generated):
  real_loss = loss_obj(tf.ones_like(real), real)

  generated_loss = loss_obj(tf.zeros_like(generated), generated)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss * 0.5

In [ ]:
def generator_loss(generated):
  return loss_obj(tf.ones_like(generated), generated)

Consistência do ciclo significa que o resultado deve ficar próximo da entrada original. Por exemplo, se uma frase for traduzida do inglês para o francês e depois traduzida de volta do francês para o inglês, a frase resultante deve ser a mesma que a original.

Na perda de consistência do ciclo:

- A imagem $X$ é passada pelo gerador $G$, que gera a imagem gerada $\hat{Y}$.
- A imagem gerada $\hat{Y}$ é passada pelo gerador $F$, que gera a imagem que passou pelo ciclo $\hat{X}$.
- O erro absoluto médio é calculado entre $X$ e $\hat{X}$.

$$forward\ cycle\ consistency\ loss: X -> G(X) -> F(G(X)) \sim \hat{X}$$

$$backward\ cycle\ consistency\ loss: Y -> F(Y) -> G(F(Y)) \sim \hat{Y}$$

![Perda do ciclo](https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/generative/images/cycle_loss.png?raw=true)

In [ ]:
def calc_cycle_loss(real_image, cycled_image):
  loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))
  
  return LAMBDA * loss1

Conforme mostrado acima, o gerador $G$ é responsável por transformar a imagem $X$ na imagem $Y$. A perda de identidade diz que, se você alimentar a imagem $Y$ no gerador $G$, ele deve gerar a imagem real $Y$ ou algo próximo da imagem $Y$.

Se você executar o modelo “zebra para cabalo” em um cavalo ou o modelo “cavalo para zebra” em uma zebra, ele não deve modificar a imagem muito, já que ela já contém a classe alvo.

$$Identity\ loss = |G(Y) - Y| + |F(X) - X|$$

In [ ]:
def identity_loss(real_image, same_image):
  loss = tf.reduce_mean(tf.abs(real_image - same_image))
  return LAMBDA * 0.5 * loss

Inicialize os otimizadores para todos os geradores e discriminadores.

In [ ]:
generator_g_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
generator_f_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

discriminator_x_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_y_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

## Checkpoints

In [ ]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(generator_g=generator_g,
                           generator_f=generator_f,
                           discriminator_x=discriminator_x,
                           discriminator_y=discriminator_y,
                           generator_g_optimizer=generator_g_optimizer,
                           generator_f_optimizer=generator_f_optimizer,
                           discriminator_x_optimizer=discriminator_x_optimizer,
                           discriminator_y_optimizer=discriminator_y_optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

## Treinamento

Observação: o modelo deste exemplo é treinado com menos épocas (10) do que o artigo (200) para manter o tempo de treinamento razoável para este tutorial. As imagens geradas terão uma qualidade muito inferior.

In [ ]:
EPOCHS = 10

In [ ]:
def generate_images(model, test_input):
  prediction = model(test_input)
    
  plt.figure(figsize=(12, 12))

  display_list = [test_input[0], prediction[0]]
  title = ['Input Image', 'Predicted Image']

  for i in range(2):
    plt.subplot(1, 2, i+1)
    plt.title(title[i])
    # getting the pixel values between [0, 1] to plot it.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

Embora o loop de treinamento pareça complicado, ele consiste de quatro passos básicos:

- Obter as previsões.
- Calcular a perda.
- Calcular os gradientes usando retropropagação.
- Aplicar os gradientes ao otimizador.

In [ ]:
@tf.function
def train_step(real_x, real_y):
  # persistent is set to True because the tape is used more than
  # once to calculate the gradients.
  with tf.GradientTape(persistent=True) as tape:
    # Generator G translates X -> Y
    # Generator F translates Y -> X.
    
    fake_y = generator_g(real_x, training=True)
    cycled_x = generator_f(fake_y, training=True)

    fake_x = generator_f(real_y, training=True)
    cycled_y = generator_g(fake_x, training=True)

    # same_x and same_y are used for identity loss.
    same_x = generator_f(real_x, training=True)
    same_y = generator_g(real_y, training=True)

    disc_real_x = discriminator_x(real_x, training=True)
    disc_real_y = discriminator_y(real_y, training=True)

    disc_fake_x = discriminator_x(fake_x, training=True)
    disc_fake_y = discriminator_y(fake_y, training=True)

    # calculate the loss
    gen_g_loss = generator_loss(disc_fake_y)
    gen_f_loss = generator_loss(disc_fake_x)
    
    total_cycle_loss = calc_cycle_loss(real_x, cycled_x) + calc_cycle_loss(real_y, cycled_y)
    
    # Total generator loss = adversarial loss + cycle loss
    total_gen_g_loss = gen_g_loss + total_cycle_loss + identity_loss(real_y, same_y)
    total_gen_f_loss = gen_f_loss + total_cycle_loss + identity_loss(real_x, same_x)

    disc_x_loss = discriminator_loss(disc_real_x, disc_fake_x)
    disc_y_loss = discriminator_loss(disc_real_y, disc_fake_y)
  
  # Calculate the gradients for generator and discriminator
  generator_g_gradients = tape.gradient(total_gen_g_loss, 
                                        generator_g.trainable_variables)
  generator_f_gradients = tape.gradient(total_gen_f_loss, 
                                        generator_f.trainable_variables)
  
  discriminator_x_gradients = tape.gradient(disc_x_loss, 
                                            discriminator_x.trainable_variables)
  discriminator_y_gradients = tape.gradient(disc_y_loss, 
                                            discriminator_y.trainable_variables)
  
  # Apply the gradients to the optimizer
  generator_g_optimizer.apply_gradients(zip(generator_g_gradients, 
                                            generator_g.trainable_variables))

  generator_f_optimizer.apply_gradients(zip(generator_f_gradients, 
                                            generator_f.trainable_variables))
  
  discriminator_x_optimizer.apply_gradients(zip(discriminator_x_gradients,
                                                discriminator_x.trainable_variables))
  
  discriminator_y_optimizer.apply_gradients(zip(discriminator_y_gradients,
                                                discriminator_y.trainable_variables))

In [ ]:
for epoch in range(EPOCHS):
  start = time.time()

  n = 0
  for image_x, image_y in tf.data.Dataset.zip((train_horses, train_zebras)):
    train_step(image_x, image_y)
    if n % 10 == 0:
      print ('.', end='')
    n += 1

  clear_output(wait=True)
  # Using a consistent image (sample_horse) so that the progress of the model
  # is clearly visible.
  generate_images(generator_g, sample_horse)

  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))

  print ('Time taken for epoch {} is {} sec\n'.format(epoch + 1,
                                                      time.time()-start))

## Gerar usando o dataset de teste

In [ ]:
# Run the trained model on the test dataset
for inp in test_horses.take(5):
  generate_images(generator_g, inp)

## Próximos passos

Este tutorial mostrou como implementar uma CycleGAN começando pelo gerador e discriminador implementados no tutorial do [Pix2Pix](https://www.tensorflow.org/tutorials/generative/pix2pix). Agora, você pode tentar usar um dataset diferente disponível nos [TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/cycle_gan).

Além disso, você pode fazer o treinamento usando um número maior de épocas para melhorar os resultados ou pode implementar o gerador ResNet modificado usado no [artigo](https://arxiv.org/abs/1703.10593) em vez do gerador U-Net usado aqui.